In [5]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import sklearn.utils.validation

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score
# [핵심 변경 1] 일반 SMOTE 대신 경계선 데이터를 만드는 BorderlineSMOTE 사용
from imblearn.over_sampling import BorderlineSMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# 1. 데이터 로드
print("딥러닝 데이터 준비 중...")
df = pd.read_csv('data/spotify_churn_dataset.csv')

if 'user_id' in df.columns:
    df = df.drop(columns=['user_id'])

# 2. Feature Engineering (ML과 동일하게 적용)
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']

# 3. 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. 전처리
numerical_cols = [
    'age', 'listening_time', 'songs_played_per_day', 'skip_rate', 
    'ads_listened_per_week', 'offline_listening',
    'ad_burden', 'satisfaction_score', 'time_per_song'
]
categorical_cols = ['gender', 'country', 'subscription_type', 'device_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# [핵심 변경 2] BorderlineSMOTE 적용
# 헷갈리는 데이터 위주로 학습 데이터를 생성하여 딥러닝 모델의 판단력을 높임
print("BorderlineSMOTE 적용 중... (시간이 조금 걸릴 수 있습니다)")
bsmote = BorderlineSMOTE(random_state=42, kind='borderline-1')
X_train_res, y_train_res = bsmote.fit_resample(X_train_processed, y_train)

# 5. 모델 설계 (Deep & Wide 구조)
input_dim = X_train_res.shape[1]
model = Sequential([
    Dense(512, input_dim=input_dim, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(256, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(128, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(64, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(32, activation='swish'),
    BatchNormalization(), Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# 6. 학습 시작
print("딥러닝 학습 시작...")
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=0)
checkpoint = ModelCheckpoint('models/spotify_dl_model.h5', monitor='val_accuracy', save_best_only=True, verbose=0)

history = model.fit(
    X_train_res, y_train_res, epochs=150, batch_size=64, validation_split=0.2,
    callbacks=[early_stop, reduce_lr, checkpoint], verbose=1
)

# 7. [핵심 변경 3] 최적의 임계값(Threshold) 찾기
# 딥러닝은 확률값을 뱉어내므로, 0.5가 아닌 최적의 기준선을 찾아 F1 점수를 극대화함
print("최적의 임계값 탐색 중...")
best_model = tf.keras.models.load_model('models/spotify_dl_model.h5')
y_pred_proba = best_model.predict(X_test_processed).flatten()

thresholds = np.arange(0.3, 0.7, 0.01)
best_thresh = 0.5
best_f1 = 0
best_acc = 0

for thresh in thresholds:
    y_pred_temp = (y_pred_proba >= thresh).astype(int)
    f1_temp = f1_score(y_test, y_pred_temp)
    
    if f1_temp > best_f1:
        best_f1 = f1_temp
        best_acc = accuracy_score(y_test, y_pred_temp)
        best_thresh = thresh

# 최종 결과 출력
print("-" * 40)
print(f"🚀 딥러닝 최적 임계값 발견: {best_thresh:.2f}")
print(f"최종 Accuracy: {best_acc:.4f}")
print(f"최종 F1-Score: {best_f1:.4f}")
print("-" * 40)

# 8. 저장
if not os.path.exists('models'): os.makedirs('models')
joblib.dump(preprocessor, 'models/dl_preprocessor.pkl')

metrics_file = 'data/model_metrics.json'
final_metrics = {}

if os.path.exists(metrics_file):
    try:
        with open(metrics_file, 'r') as f:
            final_metrics = json.load(f)
    except:
        pass

# 메트릭 업데이트
final_metrics['Deep Learning (DNN)'] = {
    'Accuracy': best_acc, 
    'F1-Score': best_f1,
    'Best Threshold': float(best_thresh)
}

with open(metrics_file, 'w') as f:
    json.dump(final_metrics, f, indent=4)

print("딥러닝 모델 및 점수 저장 완료.")

딥러닝 데이터 준비 중...
BorderlineSMOTE 적용 중... (시간이 조금 걸릴 수 있습니다)
딥러닝 학습 시작...
Epoch 1/150
126/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6106 - loss: 0.7981

137/137 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6605 - loss: 0.6995 - val_accuracy: 4.5809e-04 - val_loss: 0.9426 - learning_rate: 5.0000e-04
Epoch 2/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7322 - loss: 0.5527

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7312 - loss: 0.5483 - val_accuracy: 0.0779 - val_loss: 0.9571 - learning_rate: 5.0000e-04
Epoch 3/150
126/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7520 - loss: 0.5221

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7512 - loss: 0.5151 - val_accuracy: 0.3798 - val_loss: 0.8083 - learning_rate: 5.0000e-04
Epoch 4/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7571 - loss: 0.4999

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7595 - loss: 0.4944 - val_accuracy: 0.5328 - val_loss: 0.7265 - learning_rate: 5.0000e-04
Epoch 5/150
128/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7685 - loss: 0.4822

...

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7730 - loss: 0.4506 - val_accuracy: 0.8443 - val_loss: 0.6287 - learning_rate: 5.0000e-04
Epoch 13/150
129/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7813 - loss: 0.4453

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7806 - loss: 0.4465 - val_accuracy: 0.8694 - val_loss: 0.5949 - learning_rate: 5.0000e-04
Epoch 14/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7788 - loss: 0.4431 - val_accuracy: 0.8429 - val_loss: 0.6128 - learning_rate: 5.0000e-04
Epoch 15/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7815 - loss: 0.4431 - val_accuracy: 0.8273 - val_loss: 0.6423 - learning_rate: 5.0000e-04
Epoch 16/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7781 - loss: 0.4455 - val_accuracy: 0.8369 - val_loss: 0.6301 - learning_rate: 5.0000e-04
Epoch 17/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7863 - loss: 0.4361 - val_accuracy: 0.8617 - val_loss: 0.6087 - learning_rate: 5.0000e-04
Epoch 18/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7775 - loss: 0.4379 - val_accuracy: 0.8676 - val_loss: 0.6001 - learning_rate: 5.0000e-04
Epoch 19/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy:

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7894 - loss: 0.4253 - val_accuracy: 0.8708 - val_loss: 0.5914 - learning_rate: 2.5000e-04
Epoch 24/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7858 - loss: 0.4318 - val_accuracy: 0.8612 - val_loss: 0.6002 - learning_rate: 2.5000e-04
Epoch 25/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7888 - loss: 0.4281 - val_accuracy: 0.8575 - val_loss: 0.6007 - learning_rate: 2.5000e-04
Epoch 26/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7862 - loss: 0.4288 - val_accuracy: 0.8465 - val_loss: 0.6087 - learning_rate: 2.5000e-04
Epoch 27/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7895 - loss: 0.4317 - val_accuracy: 0.8585 - val_loss: 0.5974 - learning_rate: 2.5000e-04
Epoch 28/150
129/137 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7742 - loss: 0.4309

137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7848 - loss: 0.4254 - val_accuracy: 0.8818 - val_loss: 0.5756 - learning_rate: 2.5000e-04
Epoch 29/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7864 - loss: 0.4259 - val_accuracy: 0.8525 - val_loss: 0.5975 - learning_rate: 2.5000e-04
Epoch 30/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7859 - loss: 0.4264 - val_accuracy: 0.8612 - val_loss: 0.5890 - learning_rate: 2.5000e-04
Epoch 31/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7846 - loss: 0.4276 - val_accuracy: 0.8443 - val_loss: 0.6054 - learning_rate: 2.5000e-04
Epoch 32/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7881 - loss: 0.4212 - val_accuracy: 0.8672 - val_loss: 0.5853 - learning_rate: 2.5000e-04
Epoch 33/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7893 - loss: 0.4239 - val_accuracy: 0.8768 - val_loss: 0.5790 - learning_rate: 2.5000e-04
Epoch 34/150
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy:

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
----------------------------------------
🚀 딥러닝 최적 임계값 발견: 0.53
최종 Accuracy: 0.8195
최종 F1-Score: 0.7505
----------------------------------------
딥러닝 모델 및 점수 저장 완료.
